In [3]:
import py3Dmol
import copy
import itertools
from rdkit import Chem
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem
from rdkit.Chem import rdBase
from rdkit.Chem import rdMolAlign
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.EnumerateStereoisomers import EnumerateStereoisomers, StereoEnumerationOptions
import numpy as np
import itertools
import os
p = AllChem.ETKDGv2()
p.randomSeed = 42
p.verbose = False

#based on 
https://iwatobipen.wordpress.com/2018/09/26/3d-alignment-function-of-rdkit-rdkit/ 
https://www.rdkit.org/UGM/2012/Ebejer_20110926_RDKit_1stUGM.pdf
http://rdkit.blogspot.com/2019/01/more-on-constrained-embedding.html
http://rdkit.blogspot.com/2013/12/using-allchemconstrainedembed.html

In [4]:
def optimize_conformation(mol, addHs=True):
    """optimize conformation of molecule 
    mol: is rdkit molecule object
    addHs: whether to add hydrogens or not
    """
    if addHs:
        mol = Chem.AddHs(mol)
        
    AllChem.EmbedMolecule(mol,randomSeed=42)
    AllChem.MMFFOptimizeMolecule(mol) 

    return mol

In [5]:
def get_mcs(mylist):
    """Get MCS of molecules in list, input should be a list"""
    # calc MCS
    res=rdFMCS.FindMCS(mylist,threshold=0.9, matchValences=True, ringMatchesRingOnly=True, completeRingsOnly=True)
    p = Chem.MolFromSmarts(res.smartsString)    
    # check if all ligands match the mcs
    matchingMols = [x for x in mylist if x.HasSubstructMatch(p)]
    if len(matchingMols) != len(mylist):
        print('Warning: not all molecules contain the MCS')
    core = AllChem.DeleteSubstructs(AllChem.ReplaceSidechains(Chem.RemoveHs(matchingMols[1]),p),Chem.MolFromSmiles('*'))
    core.UpdatePropertyCache()

    return core

In [6]:
def align_conformers(ref, mol, core, addHs=True):
    """Align and score conformers of mol to reference, returns best pair
    ref: is rdkit molecule of template / reference
    mol: is rdkit molecule object
    core: is MCS"""
    # get indices of reference molecule’s atoms that match core
    ref_match = ref.GetSubstructMatch(core)

    if addHs:
        mol = Chem.AddHs(mol)

    # get mapping of attom id & it's 3d coords
    mol_match = mol.GetSubstructMatch(core)
    cmap = {mol_match[i]:ref.GetConformer().GetAtomPosition(ref_match[i]) for i in range(len(mol_match))}

    #create multiple conformers with coordmap as restriction
    AllChem.EmbedMultipleConfs(mol, 50, randomSeed=42, coordMap=cmap)

    # align molecule to reference and keep best alignment
    crippen_contribs = rdMolDescriptors._CalcCrippenContribs(mol)
    crippen_ref = rdMolDescriptors._CalcCrippenContribs(ref)

    tempscore = []
    ids = []
    for prb_cid in range(mol.GetNumConformers()):
        crippenO3A = rdMolAlign.GetCrippenO3A(prbMol = mol, refMol = ref, prbCrippenContribs = crippen_contribs,
                                            refCrippenContribs = crippen_ref, prbCid = prb_cid, refCid = 0)
        crippenO3A.Align()
        tempscore.append(crippenO3A.Score())
        ids.append([prb_cid])
    best = np.argmax(tempscore)
    # select best isomer
    best_mol = Chem.MolFromMolBlock(Chem.MolToMolBlock(mol, confId=int(ids[best][0])), removeHs=False)

    return ref, best_mol

In [7]:
def main(ref, intermediate):
    """Code for optimizing the conformation, generating conformers and aligning conformers to reference."""

    # get lowest energy conformation of template
 #   ref = optimize_conformation(ref, addHs=True)

    # get MCS core
    core = get_mcs([intermediate, ref])
    # align to reference
    ref, intermediate_new = align_conformers(ref, intermediate, core)

    return ref, intermediate_new

In [9]:
mol = Chem.rdmolfiles.MolFromPDBFile('/home/willem/projects/CCR2/pymemdyn/local_new/LIG_49256B.pdb')


In [14]:
# load molecules
sdf = False
#ligands  = [m for m in Chem.SDMolSupplier('Reference_pantelis_1215_1.sdf')]
ligands = Chem.rdmolfiles.MolFromPDBFile('/home/willem/projects/CCR2/pymemdyn/local_new/LIG_49256B.pdb')
#intermediates = [m for m in Chem.SDMolSupplier('iso_molecules_part_2_1.sdf')]
intermediatefile = '/home/willem/projects/CCR2/Q/ligprep/congeneric_series_aligned/prms/congeneric_series_aligned.sdf'
intermediates = [m for m in Chem.SDMolSupplier(intermediatefile)]
#ligand is your single reference ligand in sdf format. 
# Intermediates are the ligands that you would like to align to your reference ligand. 

for i, intermediate in enumerate(intermediates):
    #if i != 0: continue
    os.makedirs(f'3d4', exist_ok = True)
    # first entry in list of ligands is reference, second ligand from pair will be aligned to this
    if type(ligands) == list:
        ref = ligands[0]
    else:
        ref = ligands
    
    if mol.HasProp("_Name") == 0:
        ref.SetProp("_Name","reference")    
    ref, intermediate = main(ref, intermediate)

    # write to file
    if sdf:

        with Chem.SDWriter(f'3d4/{intermediate.GetProp("_Name")}.sdf') as w:
            intermediate.SetProp('_Name', intermediate.GetProp("_Name"))
            w.write(intermediate)    

        with Chem.SDWriter(f'3d4/{ref.GetProp("_Name")}.sdf') as w:
            ref.SetProp('_Name', ref.GetProp("_Name"))
            w.write(ref)

    with Chem.PDBWriter(f'3d4/{intermediate.GetProp("_Name")}.pdb') as w:
        w.write(intermediate)

    with Chem.PDBWriter(f'3d4/{ref.GetProp("_Name")}.pdb') as w:
        w.write(ref)


In [15]:
p_crippen = py3Dmol.view(width=600, height=400)
p_crippen.addModel(Chem.MolToMolBlock(intermediate), 'sdf')
#p_crippen.addModel(Chem.MolToMolBlock(to_align), 'sdf')
#p_crippen.addModel(Chem.MolToMolBlock(ref), 'sdf')
p_crippen.setStyle({'stick':{}})
p_crippen.render()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol